In [1]:
import numpy as np

In [2]:
Date = np.array(
    [
        2022.10,
        2022.11,
        2022.12,
        2023.01,
        2023.02,
        2023.03,
        2023.04,
        2023.05,
        2023.06,
        2023.07,
        2023.08,
        2023.09,
        2023.10,
        2023.11,
    ]
)
Prod_n = np.array(["MouTai", "Gold", "NVIDIA"])
Price = np.array(
    [
        [251.06, 1672.0, 120.76],
        [209.14, 1676.6, 141.56],
        [229.69, 1815.2, 171.35],
        [243.66, 1846.1, 143.15],
        [259.85, 1942.8, 209.43],
        [258.95, 1845.4, 226.98],
        [253.81, 2000.4, 279.65],
        [246.48, 1992.2, 289.10],
        [230.41, 1995.5, 397.70],
        [242.83, 1929.5, 424.13],
        [266.06, 1978.8, 465.07],
        [260.71, 1967.1, 485.09],
        [250.28, 1847.2, 447.82],
        [250.84, 1987.5, 423.25],
    ]
)

In [3]:
R = np.diff(Price, axis=0) / Price[:-1]

In [4]:
p = np.mean(R, axis=0)
Sigma = np.cov(R, rowvar=False)

# (1)
